In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [6]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
    """
    

    V = np.zeros(env.nS)
    policy = np.zeros([env.nS, env.nA])

    # Implement!
    V = V.reshape((-1, 1))
    
    R_a = np.array([[env.P[s][a][0][2]
                     for s in range(env.nS)]
                    for a in range(env.nA)])  # R_a : AxS
    
    P_a = np.zeros((env.nA, env.nS, env.nS))
    
    for curr_state in range(env.nS):
        for a in range(env.nA):
            for prob, next_state, reward, _ in env.P[curr_state][a]:
                P_a[a, curr_state, next_state] += prob
    
    while True:
        V_new = np.max(R_a + discount_factor*(np.matmul(P_a, V).reshape(env.nA, env.nS)),
                       axis=0).reshape((-1, 1))
        if np.max(np.abs(V_new - V)) < theta:
            break
        V = V_new
    
    V = V.flatten()
    for curr_state in range(env.nS):
        best_action = np.argmax([V[env.P[curr_state][a][0][1]] for a in range(env.nA)])
        policy[curr_state][best_action] = 1.
                
    return policy, V

In [7]:
policy, v = value_iteration(env)

print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [8]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

In [9]:
import timeit
n = 100
t = timeit.timeit("value_iteration(env)", globals=globals(), number=n) / n
print("avg time: {}".format(t))

avg time: 0.0004151941102463752


In [27]:
env.reset()
env._render()
while True:
    a = np.argmax(policy[env.s])
    obs, reward, done, info = env.step(a)
    print("-"*30)
    env._render()
    if done:
        break

T  o  o  o
o  o  o  o
x  o  o  o
o  o  o  T
------------------------------
T  o  o  o
x  o  o  o
o  o  o  o
o  o  o  T
------------------------------
x  o  o  o
o  o  o  o
o  o  o  o
o  o  o  T
